In [14]:
import requests
import numpy
import time
import re
from bs4 import BeautifulSoup
import contextlib

In [15]:
### 電影id區間 
miss = 0
start_id = 1
end_id = 6127
tStart = time.time()

with contextlib.nested(open('data/Yahoo/miss.txt', 'w'), 
                       open('data/Yahoo/movie.txt', 'w'),
                       open('data/Yahoo/director.txt', 'w'),
                       open('data/Yahoo/performer.txt', 'w'),
                       open('data/Yahoo/ymvmodule.txt', 'w')) as (f0, f1, f2, f3, f4):   
 
    for m_id in range(start_id, end_id + 1):
        
        try:
            time.sleep(abs(numpy.random.randn()))
            url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=' + str(m_id)
            res = requests.get(url, verify=True)
            res.encoding = 'utf-8'
        
            soup = BeautifulSoup(res.text)
            item = soup.find('div', {'class':'item clearfix'})
            
            ### f0 ### 
            # 無資料頁面
            if len(item) == 13:
                miss = miss + 1
                f0.write(url)
                f0.write("\n")
                continue
            
            ### f1 ###
            #print item.find('h4').text.encode('utf-8') #中文片名 
            #print item.find('h5').text.encode('utf-8') #英文片名
    
            dta1 = []
            for p in item.findAll('p'):
                dta1.append(p.find('span', {'class':'dta'}).text)
    
            detail = []
            for d in ",".join(dta1).split(",")[0:3]:
                detail.append(d)
        
            #print detail[0].encode('utf-8') #上映日期
            #print detail[1].encode('utf-8').replace('、',',') #類型
            #print detail[2].encode('utf-8') #片長
    
            vote = soup.find('div', {'class':'vote clearfix'})
            vsum = soup.find('div', {'class':'sum'})
    
            #print re.sub('%','',vote.find("strong").text.encode('utf-8')) #想看人數(%)
            #print vote.find("q").text.encode('utf-8') #投票人數
            #print vsum.find("em").text.encode('utf-8') #總評分
            #print vsum.find("q").text.encode('utf-8') #評分人數
    
            f1.write(str(m_id)+";"+item.find('h4').text.encode('utf-8')+";"+item.find('h5').text.encode('utf-8')+";"+detail[0].encode('utf-8')+";"+detail[1].encode('utf-8').replace('、',',')+";"+detail[2].encode('utf-8')+ ";" + re.sub('%','',vote.find("strong").text.encode('utf-8')) + ";" + vote.find("q").text.encode('utf-8') + ";" + vsum.find("em").text.encode('utf-8') + ";" + vsum.find("q").text.encode('utf-8'))
            f1.write("\n")
            
            ### f2 ###
            dta2 = []
            for p2 in item.findAll('p'):
                dta2.append(p2.find('span', {'class':'dta'}))
        
            director1 = []
            for dta22 in dta2[3].findAll('a'):
                director1.append(re.sub('(\w+)|\(|\)|\.|\-', '', dta22['href'].replace('/artist_filmography.html?name=','').strip()))
            
            #print ','.join(director1) #中文導演名稱
    
            dta21 = []
            for p in item.findAll('p'):
                dta21.append(p.find('span', {'class':'dta'}).text)
    
            director2 = []
            ### 過濾中文 & 特殊符號 & 數字
            for i in ",".join(dta21).split(",")[3:-2]:
                for j in re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8'))).split(")("):
                    director2.append(re.sub('\\n', ',', re.sub('\(|\)|', '', j)).strip())
    
            #print director2[0] #英文導演名稱
            
            f2.write(str(m_id) + ";" + ','.join(director1).encode('utf-8') + ";" + director2[0])
            f2.write("\n")
            
            ### f3 ###
            dta3 = []
            for p3 in item.findAll('p'):
                dta3.append(p3.find('span', {'class':'dta'}))

            performer1 = []
            for dta3 in dta2[4].findAll('a'):
                performer1.append(re.sub('(\w+)|\(|\)|\.', '', dta3['href'].replace('/artist_filmography.html?name=','').replace('\r\n','')))

            #print ','.join(performer1) #中文演員名稱
    
            dta21 = []
            for p in item.findAll('p'):
                dta21.append(p.find('span', {'class':'dta'}).text)
    
            performer2 = []
            ### 過濾中文 & 特殊符號 & 數字
            #for i in ",".join(dta21).split(",")[3:-2]:
            #    for j in re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8'))).split(")("):
            #        performer2.append(re.sub('\\n', ',', re.sub('\(|\)|', '', j)))
                    
            ### 找出()內的英文演員名字
            for i in ",".join(dta21).split(",")[4:-2]:
                text =  re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8')))
                #print text
                m1 = re.split(r'\(\)' ,text) 
                for m2 in m1:
                    for m3 in re.sub('\)(\w+)', '', m2).split("("):
                        #print m3.replace(')','').replace(";","")
                        performer2.append(re.sub('\)|;','', m3).replace('\r\n',''))
                        
            #print ','.join(performer2[1:]) #英文演員名稱
            
            f3.write(str(m_id) + ";" + ','.join(performer1).encode('utf-8') + ";" + ','.join(performer2[:]))
            f3.write("\n")
            
            ### f4 ###
            try:
                ymvs = soup.find('div', {'id':'ymvs'}).find('div', {'class':'text full'}).find('p')
                f4.write(str(m_id)+";"+ymvs.text.encode('utf-8').replace('《','(').replace('》',')'))
                f4.write("\n")
                #print ymvs.text #劇情簡介
                
            except BaseException, e:
                ymvs = soup.find('div', {'id':'ymvs'}).find('div', {'class':'text show'}).find('p')
                f4.write(str(m_id)+";"+ymvs.text.encode('utf-8').replace('《','(').replace('》',')'))
                f4.write("\n")
            
        except BaseException, e:
            print e

print "missing: " + str(miss)
print "duration: " + time.strftime("%H:%M:%S", time.gmtime(time.time() - tStart))


missing: 941
duration: 03:26:32


#導演Director

In [3]:
m_id = 5501

url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=' + str(m_id)
res = requests.get(url, verify=True)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)

with open('data/Yahoo/director_test.txt', 'w') as f2:
    
    item = soup.find('div', {'class':'item clearfix'})
    
    dta2 = []
    for p2 in item.findAll('p'):
        dta2.append(p2.find('span', {'class':'dta'}))
        
    director1 = []
    for dta22 in dta2[3].findAll('a'):
        director1.append(re.sub('(\w+)|\(|\)|\.|\-', '', dta22['href'].replace('/artist_filmography.html?name=','').strip()))

    print re.sub('(\w+)|\(|\)|\.', '', ','.join(director1)).strip() #中文導演名稱
    
    dta21 = []
    for p in item.findAll('p'):
        dta21.append(p.find('span', {'class':'dta'}).text)
    
    director2 = []
    ### 過濾中文 & 特殊符號 & 數字
    for i in ",".join(dta21).split(",")[3:-2]:
        for j in re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8'))).split(")("):
            director2.append(re.sub('\\n', ',', re.sub('\(|\)|', '', j)).strip())
    
    print director2[0] #英文導演名稱
    
    f2.write(str(m_id) + ";" + ','.join(director1).encode('utf-8') + ";" + director2[0])
    f2.write("\n")

藤田容介（ ）
FUJITA Yosuke


#演員Performer

In [10]:
m_id = 5501

url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=' + str(m_id)
res = requests.get(url, verify=True)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)

with open('data/Yahoo/performer_test.txt', 'w') as f3:
    
    item = soup.find('div', {'class':'item clearfix'})
    
    ## f3 ###
    dta3 = []
    for p3 in item.findAll('p'):
        dta3.append(p3.find('span', {'class':'dta'}))

    performer1 = []
    for dta3 in dta2[4].findAll('a'):
        performer1.append(re.sub('(\w+)|\(|\)|\.', '', dta3['href'].replace('/artist_filmography.html?name=','').replace('\r\n','')))

    print re.sub('(\w+)|\(|\)', '' , ','.join(performer1).strip())#中文演員名稱
    
    dta21 = []
    for p in item.findAll('p'):
        dta21.append(p.find('span', {'class':'dta'}).text)
    
    performer2 = []
    ### 過濾中文 & 特殊符號 & 數字
    #for i in ",".join(dta21).split(",")[3:-2]:
    #    for j in re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8'))).split(")("):
    #        performer2.append(re.sub('\\n', ',', re.sub('\(|\)|', '', j)))
                    
    ### 找出()內的英文演員名字
    for i in ",".join(dta21).split(",")[4:-2]:
        text =  re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8')))
        #print text
        m1 = re.split(r'\(\)' ,text) 
        for m2 in m1:
            for m3 in re.sub('\)(\w+)', '', m2).split("("):
                #print m3.replace(')','').replace(";","")
                performer2.append(re.sub('\)|;','', m3).replace('\r\n','').strip())
                
    print ','.join(performer2[:]).strip() #英文演員名稱
            
    f3.write(str(m_id) + ";" + ','.join(performer1).encode('utf-8') + ";" + ','.join(performer2[:]))
    f3.write("\n")

大島美幸（ ）,荒川良良（ ）,水川麻美（ ）
OSHIMA MiyukiARAKAWA YoshiyoshiMIZUKAWA Asami


#電影資料 & 劇情簡介

In [37]:
m_id = 5515
url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=' + str(m_id)
res = requests.get(url, verify=True)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)

with open('data/Yahoo/movie_test.txt', 'w') as f1:
    
    item = soup.find('div', {'class':'item clearfix'})
    
    #print item.find('h4').text.encode('utf-8') #中文片名 
    #print item.find('h5').text.encode('utf-8') #英文片名
    
    dta1 = []
    for p in item.findAll('p'):
        dta1.append(p.find('span', {'class':'dta'}).text)
    
    detail = []
    for d in ",".join(dta1).split(",")[0:3]:
        detail.append(d)
        
    #print detail[0].encode('utf-8') #上映日期
    #print detail[1].encode('utf-8').replace('、',',') #類型
    #print detail[2].encode('utf-8') #片長
    
    vote = soup.find('div', {'class':'vote clearfix'})
    vsum = soup.find('div', {'class':'sum'})
    
    #print re.sub('%','',vote.find("strong").text.encode('utf-8')) #想看人數(%)
    #print vote.find("q").text.encode('utf-8') #投票人數
    #print vsum.find("em").text.encode('utf-8') #總評分
    #print vsum.find("q").text.encode('utf-8') #評分人數
    
    f1.write(str(m_id)+";"+item.find('h4').text.encode('utf-8')+";"+item.find('h5').text.encode('utf-8')+";"+detail[0].encode('utf-8')+";"+detail[1].encode('utf-8').replace('、',',')+";"+detail[2].encode('utf-8')+ ";" + re.sub('%','',vote.find("strong").text.encode('utf-8')) + ";" + vote.find("q").text.encode('utf-8') + ";" + vsum.find("em").text.encode('utf-8') + ";" + vsum.find("q").text.encode('utf-8'))

with open('data/Yahoo/ymvmodule_test.txt', 'w') as f4:
    try:
        ymvs = soup.find('div', {'id':'ymvs'}).find('div', {'class':'text full'}).find('p')
        f4.write(str(m_id)+";"+ymvs.text.encode('utf-8').replace('《','(').replace('》',')').strip())
        print (ymvs.text) #劇情簡介
        
    except BaseException, e:
        ymvs = soup.find('div', {'id':'ymvs'}).find('div', {'class':'text show'}).find('p')
        f4.write(str(m_id)+";"+ymvs.text.encode('utf-8').replace('《','(').replace('》',')').replace('\r\n',''))
    

由環球影業以及Illumination娛樂公司出品的《小小兵》故事起源於盤古開天時期，剛開始只是一個黃色的有機體，隨著時間漸漸演化成小小兵團，他們的存在目的就是要服侍當代最兇惡的主人，然而從T. Rex 恐龍，埃及法老王，拿破崙甚至是德古拉，小小兵總是不停的失敗了-到最後小小兵們發現已經沒有可以服侍的主人，從此開始鬱鬱寡歡，甚至逐漸凋零。這時其中的一隻小小兵—凱文他有個偉大的計畫，就是帶著他的麻吉—大胃王史都華還有膽小的蘿蔔，挺身而出踏上尋找新主人的冒險旅程…《小小兵》電影配樂精心搭配60年代傳唱至今的暢銷金曲，並且由Illumination娛樂公司的克里斯梅勒丹德利、珍娜海莉製作，皮耶考芬和凱爾巴爾達執導，布萊恩林區編劇和克里斯雷納德擔任執行製片。


#舊版程式

In [4]:
f0 = open("./data/Yahoo/miss.txt", "w")
f1 = open("./data/Yahoo/movie.txt", "w")
f2 = open("./data/Yahoo/director.txt", "w")
f3 = open("./data/Yahoo/performer.txt", "w")

### 電影id區間 
miss = 0
start_id = 1
end_id = 6044


for m_id in range(start_id, end_id + 1):
    #time.sleep(abs(numpy.random.randn()))
    try:
        url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=' + str(m_id)
    
        res = requests.get(url)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text)
        
        item = soup.find('div',{'class':'item clearfix'})
        vote = soup.find('div', {'class':'vote clearfix'})
        vsum = soup.find('div', {'class':'sum'})
        
        ### 跳過無資料頁面
        if len(item) == 13:
            miss = miss + 1
            f0.write(url)
            f0.write("\n")
            continue
        
        dta = []
        for p in item.findAll('p'):
            dta.append(p.find('span', {'class':'dta'}).text)
            
        ### 上映日期 & 類型 & 片長
        detail = []
        for d in ",".join(dta).split(",")[0:3]:
            detail.append(()
        
        name = []
        ### 演員過濾中文 & 特殊符號 & 數字
        for i in ",".join(dta).split(",")[3:-2]:
            for j in re.sub(r'[\x80-\xff]{3,}|(\d+)', '', str(i.encode('utf-8'))).split(")("):
                name.append(re.sub('\\n', ',', re.sub('\(|\)|', '', j)))
                #print re.sub('\(|\)', '', j)
        #print str(m_id) + ";" + name[0]
        #print str(m_id)  + ";" + ','.join(name[1:])                  
        #print item.find('h4').text.encode('utf-8') + "," + item.find('h5').text.encode('utf-8') + "," + ','.join(dta).encode('utf-8')
        #f1.write(item.find('h4').text.encode('utf-8') + ";" + item.find('h5').text.encode('utf-8') + ";" + ','.join(dta).encode('utf-8') + ";" + vote.find("strong").text.encode('utf-8') + ";" + vote.find("q").text.encode('utf-8') + ";" + vsum.find("em").text.encode('utf-8') + ";" + vsum.find("q").text.encode('utf-8'))
        f1.write(str(m_id) + ";" + item.find('h4').text.encode('utf-8') + ";" + item.find('h5').text.encode('utf-8') + ";" + vote.find("strong").text.encode('utf-8') + ";" + vote.find("q").text.encode('utf-8') + ";" + vsum.find("em").text.encode('utf-8') + ";" + vsum.find("q").text.encode('utf-8'))
        f1.write("\n")
        f2.write(str(m_id) + ";" + name[0])
        f2.write("\n")
        f3.write(str(m_id)  + ";" + ','.join(name[1:]))
        f3.write("\n")
        
    except BaseException, e:
        print e

print "missing:" + str(miss)
f0.close()
f1.close()
f2.close()
f3.close()

missing:0


#空白頁面

In [8]:
url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=9'
    
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)
item = soup.find('div',{'class':'item clearfix'})
print len(item)

13


#電影評分

In [27]:
### 復仇者聯盟2：奧創紀元
url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=5499'
    
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)

vote = soup.find('div', {'class':'vote clearfix'})
print vote.find("strong").text
print vote.find("q").text

vsum = soup.find('div', {'class':'sum'})
print vsum.find("em").text
print vsum.find("q").text

98%
909
4.3
871


#其他測試

In [183]:
### 中文演員名稱
url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=2000'    
res = requests.get(url)

m = re.search('(\/artist\_filmography\.html\?name\=.*\')', res.text)
print m.group()

/artist_filmography.html?name=張藝謀'


In [33]:
### 導演&演員名稱
url = 'https://tw.movies.yahoo.com/movieinfo_main.html/id=4928'    
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text)
item = soup.find('div',{'class':'item clearfix'})

dta = []
for p in item.findAll('p'):
    #print p.text
    dta.append(p.find('span', {'class':'dta'}).text)
print ",".join(dta)

name = []
### 過濾中文 & 特殊符號
for i in ",".join(dta).split(",")[3:-2]:
    for j in re.sub(r'[\x80-\xff]{3,}', '', str(i.encode('utf-8'))).split(")("):
        name.append(re.sub('\(|\)', '', j))
        #print re.sub('\(|\)', '', j)
print name[1:]
print ','.join(name[1:])
        
### 過濾英文
for i in ",".join(dta).split(",")[3:-2]:
    print re.sub(r'(\w+)|\(|\)', '', str(i.encode('utf-8')))

### 過濾電影名稱 (TBD)
for i in ",".join(dta).split(",")[3:-2]:
    m = re.search(r'([^【]+)', str(i.encode('utf-8')))
    print m.group(0)

2013-12-06,劇情,1時34分,彼德藍德斯曼(Peter Landesman),《幸運符》柴克艾弗隆 (Zac Efron)、《神秘河流》瑪西雅蓋哈登 (Marcia Gay Harden)、《世界末日》比利鮑伯松頓 (Billy Bob Thornton)、《派特的幸福劇本》賈姬維佛 (Jacki Weaver)、《醉後大丈夫2》保羅賈麥提(Paul Giamatti)、《鬼霧》湯姆威林(Tom Welling)、《鋼鐵人3》詹姆斯巴吉戴爾 (James Badge Dale)、《林肯》傑奇厄爾海利 (Jackie Earl Haley)、《金剛》柯林漢克斯( Colin Hanks)、《真愛零距離》朗李文斯頓(Ron Livingston),CatchPlay,http://www.catchplay.com/parklandhttps://www.facebook.com/catchplay
[' Zac Efron Marcia Gay Harden Billy Bob Thornton Jacki Weaver2Paul Giamatti', 'Tom Welling3 James Badge Dale Jackie Earl Haley', ' Colin Hanks', 'Ron Livingston']
 Zac Efron Marcia Gay Harden Billy Bob Thornton Jacki Weaver2Paul Giamatti,Tom Welling3 James Badge Dale Jackie Earl Haley, Colin Hanks,Ron Livingston
彼德藍德斯曼 
《幸運符》柴克艾弗隆  、《神秘河流》瑪西雅蓋哈登   、《世界末日》比利鮑伯松頓   、《派特的幸福劇本》賈姬維佛  、《醉後大丈夫》保羅賈麥提 、《鬼霧》湯姆威林 、《鋼鐵人》詹姆斯巴吉戴爾   、《林肯》傑奇厄爾海利   、《金剛》柯林漢克斯  、《真愛零距離》朗李文斯頓 
彼德藍德斯曼(Peter Landesman)
�幸運符


In [322]:
import re
cleanCom1 = r'\"()"'
text = "【限制級戰警】羅伯柯恩(Rob Cohen);保羅沃克(Paul Walker)、【限制級戰警】馮迪索(Vin Diesel)、【反恐特警組】蜜雪兒羅卓奎茲(Michelle Rodriguez)、裘丹娜布魯斯特(Jordana Brewster)、查德林伯(Chad Lindberg)"
print text
text = re.sub(r'[\x80-\xff]{3,}', '', text) #清除所有中文字 & 特殊符號
for i in text.split(")("):
    print re.sub('\(|\)', '', i)

【限制級戰警】羅伯柯恩(Rob Cohen);保羅沃克(Paul Walker)、【限制級戰警】馮迪索(Vin Diesel)、【反恐特警組】蜜雪兒羅卓奎茲(Michelle Rodriguez)、裘丹娜布魯斯特(Jordana Brewster)、查德林伯(Chad Lindberg)
Rob Cohen;Paul Walker
Vin Diesel
Michelle Rodriguez
Jordana Brewster
Chad Lindberg


In [343]:
import re
cleanCom1 = r'\"()"'

text = "【永不妥協】茱莉亞羅勃茲(Julia Roberts)、【芝加哥】凱薩琳麗塔瓊斯(Catherine Zeta-Jones)、【致命ID】約翰庫薩克(John Cusack)、比利克里斯多(Billy Crystal)"
#text = "【限制級戰警】羅伯柯恩(Rob Cohen);保羅沃克(Paul Walker)、【限制級戰警】馮迪索(Vin Diesel)、【反恐特警組】蜜雪兒羅卓奎茲(Michelle Rodriguez)、裘丹娜布魯斯特(Jordana Brewster)、查德林伯(Chad Lindberg)"

print text

text = re.sub(r'[\x80-\xff]{3,}', '', text)
print text

m1 = re.split(r'\(\)' ,text) #找出()內的英文演員名字
for m2 in m1:
    for m3 in re.sub('\)(\w+)', '', m2).split("("):
        print m3.replace(')','').replace(";","")

【永不妥協】茱莉亞羅勃茲(Julia Roberts)、【芝加哥】凱薩琳麗塔瓊斯(Catherine Zeta-Jones)、【致命ID】約翰庫薩克(John Cusack)、比利克里斯多(Billy Crystal)
(Julia Roberts)(Catherine Zeta-Jones)ID(John Cusack)(Billy Crystal)

Julia Roberts
Catherine Zeta-Jones
John Cusack
Billy Crystal


In [306]:
type(text)

str